In [2]:
#DBOW DOC2VEC model

import logging
import pymysql
import multiprocessing
import gensim
from gensim.models import Doc2Vec

from google.colab import drive
drive.mount('/gdrive')

from multiprocessing import cpu_count
cpu_count = cpu_count()
print(f"Количество потоков: {cpu_count}")

import numpy as np

# Connect to the database
def get_connection():
    connection = pymysql.connect(host='database-3.chim8btj05zt.us-east-1.rds.amazonaws.com',
                                 user='admin',
                                 password='01035007',
                                 db='test1',
                                 charset='utf8mb4',
                                 cursorclass=pymysql.cursors.DictCursor)
    return connection

# Create the tagged document needed for Doc2Vec
def create_tagged_document(list_of_list_of_words):
    for i, list_of_words in enumerate(list_of_list_of_words):
        yield gensim.models.doc2vec.TaggedDocument(list_of_words, [i])


connection = get_connection()

EPOCHS_0 = 20 #эпохи первичное обучение
EPOCHS_1 = 20 #эпохи дообучение
COUNT_DB = 10314497 #число строк в обработанной БД
LIMIT_0 = 2000000 #загружаем столько строк для первичного обучения модели
LIMIT_1 = LIMIT_0  #загружаем столько строк для последующего обучения модели

SIZE = 1000 #размерность выходного вектора признаков
ITER = 1 #C какой итерации дообучать (0 - первичное обучение)

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
try:
    with connection.cursor() as cursor:
        if ITER == 0:
            #вычисление конечного id для ITER_0
            s = "SELECT max(id) FROM (SELECT id from sentiment_v3 ORDER BY id ASC LIMIT %s) AS t"
            cursor.execute(s, (LIMIT_0))
            end_id = cursor.fetchone()['max(id)']
            print(end_id)

            #вычисление максимального id для всей БД
            s = "SELECT max(id) FROM sentiment_v3"
            cursor.execute(s)
            max_id = cursor.fetchone()['max(id)']

            #загрузка данных для iter_0
            s = "SELECT ttext_stem FROM sentiment_v3 ORDER BY id ASC LIMIT %s"
            cursor.execute(s, (LIMIT_0))
            result = cursor.fetchall()
            result = [doc['ttext_stem'].split('|') for doc in result]
            print(f"Считано {len(result)} строк из базы данных")

            train_data = list(create_tagged_document(result))
            result = None

            #instantiate our DM and DBOW models
            model = Doc2Vec(min_count=5, window=5,  vector_size=SIZE, sample=1e-3, negative=5, dm=0, workers=cpu_count, dbow_words=0) #dm = 0  DBOW_model
            
            #build vocab over all reviews
            model.build_vocab(train_data, update = False)

            #train
            model.train(documents=train_data, epochs=EPOCHS_0, total_examples=model.corpus_count)
            model.save("/gdrive/My Drive/Colab Notebooks/d2v_dbow_1000/model_dbow1000_iter_0.doc2vec", sep_limit = 100 * 1024 ** 2)
            #model.delete_temporary_training_data()
          
            print(f"Модель model_dm записана на гугл диск")
            COUNT_DB -= LIMIT_0
            print(f"Осталось обработать {COUNT_DB} записей!")
            ITER += 1

        else:
            #вычисление конечного id для ITER_X
            s = "SELECT max(id) FROM (SELECT id from sentiment_v3 ORDER BY id ASC LIMIT %s) AS t"
            cursor.execute(s, (LIMIT_0 + LIMIT_1 * (ITER - 1)))
            end_id = cursor.fetchone()['max(id)']
            print(end_id)

            #вычисление максимального id для всей БД
            s = "SELECT max(id) FROM sentiment_v3"
            cursor.execute(s)
            max_id = cursor.fetchone()['max(id)']



        
        # дообучение
        while end_id < max_id:
            
            print(f"Итерация {ITER}:")
            model_old_pach = "/gdrive/My Drive/Colab Notebooks/d2v_dbow_1000/model_dbow1000_iter_" + str(ITER - 1) + ".doc2vec"
            model_pach = "/gdrive/My Drive/Colab Notebooks/d2v_dbow_1000/model_dbow1000_iter_" + str(ITER) + ".doc2vec"


            #загрузка данных для iter_X
            s = "SELECT ttext_stem FROM sentiment_v3 WHERE id > %s ORDER BY id ASC LIMIT %s"
            cursor.execute(s, (end_id, LIMIT_1))
            result = cursor.fetchall()
            result = [doc['ttext_stem'].split('|') for doc in result]
            print(f"Считано {len(result)} строк из базы данных")

            #вычисление конечного id для ITER_X
            s = "SELECT max(id) FROM (SELECT id FROM sentiment_v3 WHERE id > %s ORDER BY id ASC LIMIT %s) AS t"
            cursor.execute(s, (end_id, LIMIT_1))
            end_id = cursor.fetchone()['max(id)']

            train_data = list(create_tagged_document(result))
            result = None

            #instantiate our DM and DBOW models

            model = Doc2Vec.load(model_old_pach) #, mmap='r')  
            
            #build vocab over all reviews
            model.build_vocab(train_data, update = True)
            
            #train
            model.train(documents=train_data, epochs=EPOCHS_1, total_examples=model.corpus_count)

            #model.delete_temporary_training_data()

            model.save(model_pach, sep_limit = 100 * 1024 ** 2)
            print(f"Модель записана на гугл диск")

            COUNT_DB -= LIMIT_1
            print(f"Осталось обработать {COUNT_DB} записей!")

            ITER += 1
finally:
    connection.close()




Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
Количество потоков: 2
411050803860733952
Итерация 1:
Считано 2000000 строк из базы данных


2020-07-24 07:41:05,451 : INFO : loading Doc2Vec object from /gdrive/My Drive/Colab Notebooks/d2v_dbow_1000/model_dbow1000_iter_0.doc2vec
/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:254: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


EOFError: ignored

In [ ]:
#Обновление модели doc2vec
import logging
import pymysql
import multiprocessing
import gensim
from gensim.models import Doc2Vec

from google.colab import drive
drive.mount('/gdrive')

from multiprocessing import cpu_count
cpu_count = cpu_count()
print(f"Количество потоков: {cpu_count}")

import numpy as np

# Connect to the database
def get_connection():
    connection = pymysql.connect(host='database-3.chim8btj05zt.us-east-1.rds.amazonaws.com',
                                 user='admin',
                                 password='01035007',
                                 db='test1',
                                 charset='utf8mb4',
                                 cursorclass=pymysql.cursors.DictCursor)
    return connection

# Create the tagged document needed for Doc2Vec
def create_tagged_document(list_of_list_of_words):
    for i, list_of_words in enumerate(list_of_list_of_words):
        yield gensim.models.doc2vec.TaggedDocument(list_of_words, [i])


connection = get_connection()

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
try:
    with connection.cursor() as cursor:

        
  

finally:
    connection.close()


In [ ]:
#Предполагаемый вектор абзаца для нового документа.
print(model_dm.infer_vector(['привет', 'я', 'пришел'], epochs=100)) 
print(model_dbow.infer_vector(['привет', 'я', 'пришел']))

Параметры модели doc2vec

документы (итерируемый из списка TaggedDocument, необязательный) - входной корпус может быть просто списком элементов, но для больших корпораций рассмотрим итеративный, который передает документы непосредственно с диска / сети. Если вы не предоставите документы (или файл corpus_file ), модель останется неинициализированной - используйте, если вы планируете инициализировать ее каким-либо другим способом.

corpus_file ( str , необязательно ) - путь к файлу корпуса в LineSentenceформате. Вы можете использовать этот аргумент вместо документов, чтобы повысить производительность. Необходимо передать только один из документов или аргументы corpus_file (или ни один из них, в этом случае модель не остается инициализированной). Теги документов назначаются автоматически и равны номеру строки, как в TaggedLineDocument.

dm ( {1 , 0} , необязательно ) - определяет алгоритм обучения. Если dm = 1 , используется «распределенная память» (PV-DM). В противном случае используется распределенный пакет слов (PV-DBOW).

vector_size ( int , необязательный ) - размерность векторов объектов.

window ( int , необязательный ) - максимальное расстояние между текущим и прогнозируемым словом в предложении.

альфа (с плавающей точкой , необязательно ) - начальная скорость обучения.

min_alpha ( float , необязательно ) - скорость обучения будет линейно снижаться до min_alpha в процессе обучения.

seed ( int , необязательный ) - Seed для генератора случайных чисел. Начальные векторы для каждого слова засеяны хешем конкатенации слова + str (семя) . Обратите внимание, что для полностью детерминированно-воспроизводимого прогона вы также должны ограничить модель одним рабочим потоком (worker = 1 ), чтобы исключить дрожание порядка в планировании потоков ОС. В Python 3 воспроизводимость между запусками интерпретатора также требует использования переменной среды PYTHONHASHSEED для управления рандомизацией хеша.

min_count ( int , необязательный ) - игнорирует все слова с общей частотой ниже этой.

max_vocab_size ( int , необязательный ) - ограничивает ОЗУ при построении словарного запаса; если есть больше уникальных слов, чем это, то удалите редкие слова. Каждые 10 миллионов типов слов требуют около 1 ГБ оперативной памяти. Установите Нет для отсутствия ограничений.

sample ( float , необязательный ) - порог для настройки того, какие высокочастотные слова случайным образом отбираются, полезный диапазон (0, 1e-5).

working ( int , необязательный ) - используйте эти многочисленные рабочие потоки для обучения модели (= более быстрое обучение на многоядерных машинах).

epochs ( int , необязательный ) - количество итераций (эпох) по всему корпусу.

hs ( {1 , 0} , необязательно ) - если 1, для обучения модели будет использоваться иерархический softmax. Если установлено значение 0, а отрицательное значение не равно нулю, будет использоваться отрицательная выборка.

отрицательный ( int , необязательный ) - если> 0, будет использоваться отрицательная выборка, int для отрицательного указывает, сколько «шумовых слов» следует нарисовать (обычно между 5-20). Если установлено значение 0, отрицательная выборка не используется.

ns_exponent ( float , необязательный ) - показатель степени, используемый для формирования отрицательного распределения выборки. Значение 1,0 выборки точно пропорционально частотам, 0,0 выборки всех слов одинаково, в то время как отрицательное значение выборки низкочастотных слов больше, чем высокочастотных слов. Популярное значение по умолчанию 0,75 было выбрано оригинальной статьей Word2Vec. Совсем недавно, в https://arxiv.org/abs/1804.04212 , Caselles-Dupré, Lesaint и Royo-Letelier предлагают, чтобы другие значения могли работать лучше для рекомендательных приложений.

dm_mean ( {1 , 0} , необязательно ) - если 0, используйте сумму векторов слова контекста. Если 1, используйте среднее. Применяется только когда dm используется в неконкатентивном режиме.

dm_concat ( {1 , 0} , необязательно ) - если 1, использовать конкатенацию векторов контекста, а не сумму / среднее; Обратите внимание, что конкатенация приводит к гораздо большей модели, так как вход больше не является размером одного (сэмплированного или арифметически сложенного) слова-вектора, а размером тега (-ов) и всех слов в контексте, связанных вместе.

dm_tag_count ( int , необязательный ) - ожидаемое постоянное количество тегов документа на документ при использовании режима dm_concat.

dbow_words ( {1 , 0} , необязательно ) - если установлено значение 1, обучает векторы слов (в режиме пропуска граммы) одновременно с обучением doc-vector DBOW; Если 0, обучаются только векторы документов (быстрее).

trim_rule ( функция , необязательно ) -

Правило обрезки словаря, указывает, должны ли определенные слова оставаться в словаре, обрезаться или обрабатываться по умолчанию (отбрасывать, если количество слов <min_count). Может быть None (будет использоваться min_count, смотрите keep_vocab_item()) или вызываться, который принимает параметры (word, count, min_count) и возвращает либо gensim.utils.RULE_DISCARD, gensim.utils.RULE_KEEPлибо gensim.utils.RULE_DEFAULT. Правило, если оно задано, используется только для сокращения словарного запаса во время текущего вызова метода и не сохраняется как часть модели.

Входные параметры имеют следующие типы:
слово (str) - слово, которое мы изучаем

count (int) - частота слова в корпусе

min_count (int) - минимальный порог счета.

обратные вызовы - список обратных вызовов, которые необходимо выполнить / запустить на определенных этапах во время обучения.